# Laboratorio 8 - Word Embeddings
## <font size=4> *CC3092 - Deep Learning y Sistemas Inteligentes*, 2024 </font>
## <font size=3 color='gray'> Samuel Chamale, Adrian Rodriguez y Daniel Gómez</font>

### <ins>1. (Modelo de Transporte)</ins>

In [30]:
using JuMP, GLPK, Printf

function solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)
    # Crear el modelo
    modelo = Model(GLPK.Optimizer)

    num_centros = length(centros)
    num_concesionarios = length(concesionarios)

    # Definir variables de decisión
    @variable(modelo, x[1:num_centros, 1:num_concesionarios] >= 0)

    # Definir la función objetivo (minimizar el costo total de transporte)
    @objective(modelo, Min,
        sum(distancias[i][j] * x[i,j] for i in 1:num_centros, j in 1:num_concesionarios) * (25/18)
    )

    # Restricciones de Oferta
    for i in 1:num_centros
        @constraint(modelo, sum(x[i,j] for j in 1:num_concesionarios) == ofertas[i])
    end

    # Restricciones de Demanda
    for j in 1:num_concesionarios
        @constraint(modelo, sum(x[i,j] for i in 1:num_centros) == demandas[j])
    end

    # Resolver el modelo
    optimize!(modelo)

    # Verificar el estado de la optimización
    println("\nResultados:\n")

    # Resultados en formato de tabla
    # Imprimir encabezado de concesionarios
    print(" " ^ 15)  # Espacio en blanco para la primera columna
    for j in 1:num_concesionarios
        print(rpad(concesionarios[j], 20))  # Imprimir cada concesionario con un ancho fijo
    end
    println()

    # Imprimir los valores de transporte con centros como filas
    for i in 1:num_centros
        print(rpad(centros[i], 15))  # Imprimir el nombre del centro
        for j in 1:num_concesionarios
            print(rpad(@sprintf("%.2f", value(x[i,j])), 20))  # Imprimir el valor del transporte con 2 decimales
        end
        println()
    end

    @printf("\nCosto Total de Transporte: \$%.2f\n", objective_value(modelo))

    if termination_status(modelo) == MOI.OPTIMAL
        println("\nSe encontró una solución óptima.")
    else
        println("\nNo se encontró una solución óptima.")
    end
end

solve_transportation_model (generic function with 1 method)

In [31]:
# Original
centros = ["Cen 1", "Cen 2", "Cen 3"]
concesionarios = ["Con 1", "Con 2", "Con 3", "Con 4", "Con 5"]
distancias = [
    [100, 150, 200, 140, 35],
    [50, 70, 60, 65, 80],
    [40, 90, 100, 150, 130]
]
ofertas = [400, 200, 150]
demandas = [100, 200, 150, 160, 140]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)


Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                100.00              0.00                160.00              140.00              
Cen 2          0.00                50.00               150.00              0.00                0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87916.67

Se encontró una solución óptima.


### <ins>2. Problema 2</ins>

#### ***a) Concesionario 5 requiere 200 automóviles***

In [32]:
# Concesionario 5 requiere 200 automóviles
demandas = [100, 200, 150, 160, 200]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)


Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                0.00                0.00                260.00              140.00              
Cen 2          0.00                150.00              150.00              -100.00             0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87222.22

No se encontró una solución óptima.


#### ***b) Centro 3 produce 200 automóviles***

In [33]:
# Centro 3 produce 200 automóviles
ofertas = [400, 200, 200]
demandas = [100, 200, 150, 160, 140]

solve_transportation_model(centros, concesionarios, distancias, ofertas, demandas)


Resultados:

               Con 1               Con 2               Con 3               Con 4               Con 5               
Cen 1          0.00                0.00                0.00                260.00              140.00              
Cen 2          0.00                150.00              150.00              -100.00             0.00                
Cen 3          100.00              50.00               0.00                0.00                0.00                

Costo Total de Transporte: $87222.22

No se encontró una solución óptima.


### <ins>3. (Modelo de Asignación)</ins>

In [48]:
using JuMP, GLPK, Printf

function solve_assignment_model(cost_matrix, rows, cols)
    # Crear el modelo
    modelo = Model(GLPK.Optimizer)

    num_rows = length(rows)
    num_cols = length(cols)

    # Definir variables de decisión
    @variable(modelo, x[1:num_rows, 1:num_cols], Bin)

    # Definir la función objetivo (minimizar el costo total)
    @objective(modelo, Min, sum(cost_matrix[i][j] * x[i,j] for i in 1:num_rows, j in 1:num_cols))

    # Restricciones de una asignación por fila
    for i in 1:num_rows
        @constraint(modelo, sum(x[i,j] for j in 1:num_cols) == 1)
    end

    # Restricciones de una asignación por columna
    for j in 1:num_cols
        @constraint(modelo, sum(x[i,j] for i in 1:num_rows) == 1)
    end

    # Resolver el modelo
    optimize!(modelo)

    # Verificar el estado de la optimización
    if termination_status(modelo) == MOI.OPTIMAL
        println("Se encontró una solución óptima.")
        println("Costo Total de Asignación: \$", objective_value(modelo))
        println("Asignaciones Óptimas:")
        for i in 1:num_rows
            for j in 1:num_cols
                if value(x[i,j]) > 0.5
                    @printf("%s → %s (\$%.2f)\n", rows[i], cols[j], cost_matrix[i][j])
                end
            end
        end
    else
        println("No se encontró una solución óptima.")
    end
end

solve_assignment_model (generic function with 1 method)

#### ***Tabla (i)***

In [49]:
println("Tabla (i):\n")

# Definir la matriz de costos
cost_matrix = [
    [3, 8, 2, 10, 3],
    [6, 5, 2, 7, 5],
    [6, 4, 2, 7, 5],
    [8, 4, 2, 3, 5],
    [7, 8, 6, 7, 7]
]

# Definir filas y columnas como Trabajadores y Tareas
trabajadores = ["W1", "W2", "W3", "W4", "W5"]
tareas = ["T1", "T2", "T3", "T4", "T5"]

println("Asignación: Filas como Trabajadores, Columnas como Tareas")
solve_assignment_model(cost_matrix, trabajadores, tareas)

# Definir filas y columnas como Tareas y Trabajadores (transposición)
tareas_as_rows = ["T1", "T2", "T3", "T4", "T5"]
trabajadores_as_cols = ["W1", "W2", "W3", "W4", "W5"]

println("\nAsignación: Filas como Tareas, Columnas como Trabajadores")
solve_assignment_model(cost_matrix, tareas_as_rows, trabajadores_as_cols)

Tabla (i):

Asignación: Filas como Trabajadores, Columnas como Tareas
Se encontró una solución óptima.
Costo Total de Asignación: $19.0
Asignaciones Óptimas:
W1 → T1 ($3.00)
W2 → T3 ($2.00)
W3 → T2 ($4.00)
W4 → T4 ($3.00)
W5 → T5 ($7.00)

Asignación: Filas como Tareas, Columnas como Trabajadores
Se encontró una solución óptima.
Costo Total de Asignación: $19.0
Asignaciones Óptimas:
T1 → W1 ($3.00)
T2 → W3 ($2.00)
T3 → W2 ($4.00)
T4 → W4 ($3.00)
T5 → W5 ($7.00)

Se encontró una solución óptima.
Costo Total de Asignación: $19.0
Asignaciones Óptimas:
W1 → T1 ($3.00)
W2 → T3 ($2.00)
W3 → T2 ($4.00)
W4 → T4 ($3.00)
W5 → T5 ($7.00)

Asignación: Filas como Tareas, Columnas como Trabajadores
Se encontró una solución óptima.
Costo Total de Asignación: $19.0
Asignaciones Óptimas:
T1 → W1 ($3.00)
T2 → W3 ($2.00)
T3 → W2 ($4.00)
T4 → W4 ($3.00)
T5 → W5 ($7.00)
